In [ ]:
#Leitura da base
inicial = spark.read.format("csv").option("header", "true").load("Documents/Titanic/train.csv")

In [ ]:
import pyspark.sql.functions as SF

In [ ]:
#UDF para tratamento nos Nomes e conseguir retornar apenas os sobrenomes
@SF.udf("String")
def sobrenome(name):
    
    if name is None:
        return None
    else: 
        aux= name.replace("\""," ")
        aux1= aux.split(",")
        return aux1[0]

In [ ]:
df1=inicial.withColumn("Sobrenome",sobrenome("Name"))

In [ ]:
# Cast da idade de String para Float
df3=df1.withColumn("age", df3["age"].cast("Float"))

In [ ]:
#Inicio de tratamento com a  coluna de idade para preencher com uma idade estimada as pessas que estavam com este dado Nulo
dfi=df3.where("Sex = 'female'").groupBy("Pclass").avg("Age")
dfi=dfi.withColumn("Sex",SF.lit("female"))

#

dfi2=df3.where("Sex = 'male'").groupBy("Pclass").avg("Age")
dfi2=dfi2.withColumn("Sex",SF.lit("male"))

In [ ]:
dfu=dfi.union(dfi2)

In [ ]:
df4=df3.join(dfu,["Pclass","Sex"], 'left')

In [ ]:
@SF.udf("Float")
def idade(id1,id2):
    if id1 is None and id2 is None :
        return None
    elif id1 is None:
        return id2
    else:
        return id1
    

In [ ]:
# criacao da nova coluna com as idades estimadas das pessoas
df5=df4.withColumn("Estimated_Age", idade("Age","avg(Age)"))

In [ ]:
# cast de string para float da coluna Fare
df6=df5.withColumn("Fare", df3["Fare"].cast("Float"))

In [ ]:
# Todas as colunas  que foram dropadas da base foi por um dos motivos abaixo:
# 1- preenchimento baixo
# 2- O dado foi transformado em outra coluna
# 3- Os dados nao eram relevantes para a base no momento,ou para o uso que vai ser feito com a base (Colunas de Nome,ticket)

df7=df6.drop("avg(Age)","Cabin")

In [ ]:
df8=df7.withColumn("SibSp", df7["SibSp"].cast("Int"))

In [ ]:
df9=df8.withColumn("Parch", df8["Parch"].cast("Int"))

In [ ]:
df10=df9.groupBy("Sobrenome").count()

In [ ]:
# um novo tratamento de string na coluna sobrenome
@SF.udf("String")
def sobrenome2(name):
    if name is None:
        return None
    else:
        return str(name.strip())

In [ ]:
df9=df9.withColumn("Sobrenome", sobrenome2("Sobrenome"))

In [ ]:
df11=df9.join(df10,["Sobrenome"],'left')

In [ ]:
# Criacao da coluna family que significa quantos parentes no geral aquela pessoa tem abordo do navio
df12=df11.withColumn("Family",(df11["count"]-1))

In [ ]:
df13=df12.drop("count","SibSp","Parch","age")

In [ ]:
df14=df13.drop("Sobrenome","Name","Ticket")

In [ ]:
df15=df14.withColumn("Sex", SF.when(SF.col("Sex")=='male',0).otherwise(1))

In [ ]:
df16=df15.withColumn("Embarked", SF.when(SF.col("Embarked")=='Q',0).otherwise(SF.col("Embarked")))

In [ ]:
df17=df16.withColumn("Embarked", SF.when(SF.col("Embarked")=='C',1).otherwise(SF.col("Embarked")))

In [ ]:
df18=df17.withColumn("Embarked", SF.when(SF.col("Embarked")=='S',2).otherwise(SF.col("Embarked")))

In [ ]:
df19 = df18.withColumn("Pclass", df18["Pclass"].cast("double"))
df19 = df19.withColumn("PassengerId", df19["PassengerId"].cast("double"))
df19 = df19.withColumn("Survived", df19["Survived"].cast("double"))
df19 = df19.withColumn("Embarked", df19["Embarked"].cast("double"))


In [ ]:
df19.describe().show(100,False)

In [ ]:
# criacao de colunas auxiliares para normalizacao
df20=df19.withColumn("Max_age",SF.lit((80.0)))
df20=df20.withColumn("Max_Family",SF.lit(8.0))
df20=df20.withColumn("Max_Embarked",SF.lit(2.0))
df20=df20.withColumn("Max_Fare",SF.lit(512.3292))
df20=df20.withColumn("Max_Survived",SF.lit(1.0))
df20=df20.withColumn("Max_PassengerId",SF.lit(891.0))
df20=df20.withColumn("Max_Sex",SF.lit(1.0))
df20=df20.withColumn("Max_Pclass",SF.lit(3.0))



df20=df20.withColumn("Min_age",SF.lit(0.42))
df20=df20.withColumn("Min_Family",SF.lit(0.0))
df20=df20.withColumn("Min_Embarked",SF.lit(0.0))
df20=df20.withColumn("Min_Fare",SF.lit(0.0000))
df20=df20.withColumn("Min_Survived",SF.lit(0.0))
df20=df20.withColumn("Min_PassengerId",SF.lit(1.0))
df20=df20.withColumn("Min_Sex",SF.lit(0.0))
df20=df20.withColumn("Min_Pclass",SF.lit(1.0))

In [ ]:
# normalizacao
df21=df20.withColumn("Pclass",(((df20["Pclass"]-(df20["Min_Pclass"]))/((df20["Max_Pclass"])-(df20["Min_Pclass"])))))
df21=df21.withColumn("Family",(((df21["Family"]-(df21["Min_Family"]))/((df21["Max_Family"])-(df21["Min_Family"])))))
df21=df21.withColumn("Fare",(((df21["Fare"]-(df21["Min_Fare"]))/((df21["Max_Fare"])-(df21["Min_Fare"])))))
df21=df21.withColumn("Estimated_Age",(((df21["Estimated_Age"]-(df21["Min_age"]))/((df21["Max_age"])-(df21["Min_age"])))))
df21=df21.withColumn("Embarked",(((df21["Embarked"]-(df21["Min_Embarked"]))/((df21["Max_Embarked"])-(df21["Min_Embarked"])))))
df21=df21.withColumn("Survived",(((df21["Survived"]-(df21["Min_Survived"]))/((df21["Max_Survived"])-(df21["Min_Survived"])))))
df21=df21.withColumn("PassengerId",(((df21["PassengerId"]-(df21["Min_PassengerId"]))/((df21["Max_PassengerId"])-(df21["Min_PassengerId"])))))
df21=df21.withColumn("Sex",(((df21["Sex"]-(df21["Min_Sex"]))/((df21["Max_Sex"])-(df21["Min_Sex"])))))

In [ ]:
df22=df21.select("Pclass","Family","Estimated_Age","Fare","Embarked","PassengerId","Sex","Survived")

In [ ]:
df22.coalesce(1).write.mode("overwrite").format("com.databricks.spark.csv").option("header", "true").option("delimiter", "\t").save("Documents/Titanic/titanic_3.csv")

In [10]:
df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("Documents/Titanic/titanic_3.csv")
#df = df.select("*").toPandas()